In [2]:
import jax.numpy as np
import numpy as onp
from functools import partial
from jax import vmap
from jax.lax import scan
from jax.lax import cond
from jax import random
from jax import jit
from jax import jacrev
from jax.lax import stop_gradient
import matplotlib.pylab as plt
import matplotlib as mpl
from sklearn import manifold, datasets
import seaborn as sns
import jax

from jax.config import config
config.update("jax_debug_nans", True)

import jax.numpy as np
from utils import MidpointNormalize, load_data
from jax import random, flatten_util, vjp, jvp, custom_vjp, jacfwd, jacrev, vmap, grad
from IFD_tsne import tsne_fwd
import jax
import matplotlib.pylab as plt
import seaborn as sns

In [4]:
X, y = load_data(75)
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(X.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
Y_star = tsne_fwd(X, y_guess)

X_flat, X_unflattener = flatten_util.ravel_pytree(X)   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star) 

===> Finding 74 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.01 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 1.2387, 50 iterations in 0.2845 sec
Iteration  100, KL divergence 1.2850, 50 iterations in 0.2777 sec
Iteration  150, KL divergence 1.3106, 50 iterations in 0.2765 sec
Iteration  200, KL divergence 1.3258, 50 iterations in 0.3581 sec
Iteration  250, KL divergence 1.4002, 50 iterations in 0.3153 sec
   --> Time elapsed: 1.51 seconds
===> Running optimization with exaggeration=1.00, lr=200.00 for 750 iterations...
Iteration   50, KL divergence 0.4311, 50 iterations in 0.3099 sec
Iteration  100, KL divergence 0.3983, 50 iterations in 0.3104 sec
Iteration  150, KL divergence 0.3656, 50 iterations in 0.2776 sec
Iteration  200, KL divergence 0.3644, 50 iterations in 0.2672 sec
Iteration 

In [5]:
# Pairwise squared Euclidean distance
def get_dists(Z):
    """
    Compute all pairwise distances of each data points z_i in Z.
    
    Params:
        Z: matrix with n rows. Each row is the z_i
        
    Return:
        All pairwise distances of each data points z_i in Z
    """

    diff = Z[:, :, None] - Z[:, :, None].T
    return (np.square(diff)).sum(1)

In [6]:
def logSoftmax(x):
    """Compute softmax for vector x."""
    max_x = np.max(x)
    exp_x = np.exp(x - max_x)
    sum_exp_x = np.sum(exp_x)
    log_sum_exp_x = np.log(sum_exp_x)
    max_plus_log_sum_exp_x = max_x + log_sum_exp_x
    log_probs = x - max_plus_log_sum_exp_x

    # Recover probs
    exp_log_probs = np.exp(log_probs)
    sum_log_probs = np.sum(exp_log_probs)
    probs = exp_log_probs / sum_log_probs
    return probs

def Hbeta(D: np.ndarray, beta=1.0):
    """
    Compute the log2(perplexity)=Entropy and the P-row (P_i) for a specific value of the
        precision=1/(sigma**2) (beta) of a Gaussian distribution. D: vector of squared Euclidean distances (without i)
    :param D: vector of length d, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: H: log2(Entropy), P: computed probabilites
    """
    # TODO: exchange by softmax as described by https://nlml.github.io/in-raw-numpy/in-raw-numpy-t-sne/
    P = np.exp(-D * beta)     # numerator of p j|i
    sumP = np.sum(P, axis=None)    # denominator of p j|i --> normalization factor
    new_P = logSoftmax(-D * beta)
    sumP += 1e-8
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    return H, new_P

def Hbeta_final(D: np.ndarray, beta=1.0):
    """
    Compute the log2(perplexity)=Entropy and the P-row (P_i) for a specific value of the
        precision=1/(sigma**2) (beta) of a Gaussian distribution. D: vector of squared Euclidean distances (without i)
    :param D: vector of length d, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: H: log2(Entropy), P: computed probabilites
    """
    # TODO: exchange by softmax as described by https://nlml.github.io/in-raw-numpy/in-raw-numpy-t-sne/
    P = np.exp(-D * beta)     # numerator of p j|i
    sumP = np.sum(P, axis=None)    # denominator of p j|i --> normalization factor
    sumP += 1e-8
    new_P = P/sumP
    return new_P

def HdiffGreaterTrue(*betas):
    beta, betamax = betas
    return beta*2

def HdiffGreaterFalse(*betas):
    beta, betamax = betas
    return (beta+betamax)/2

def HdiffSmallerTrue(*betas):
    beta, betamin = betas
    return beta/2

def HdiffSmallerFalse(*betas):
    beta, betamin = betas
    return (beta+betamin)/2

def HdiffGreater(*betas):
    beta, betamin, betamax = betas
    betamin = beta
    beta = cond((np.logical_or(betamax == np.inf, betamax == -np.inf)), HdiffGreaterTrue, HdiffGreaterFalse, *(beta, betamax))
    return beta, betamin, betamax

def HdiffSmaller(*betas):
    beta, betamin, betamax = betas
    betamax = beta
    beta = cond(np.logical_or(betamin == np.inf, betamin == -np.inf), HdiffSmallerTrue, HdiffSmallerFalse, *(beta, betamin))
    return beta, betamin, betamax

def HdiffGreaterTolerance(*betas):
    beta, betamin, betamax, Hdiff = betas
    beta, betamin, betamax = cond(Hdiff > 0, HdiffGreater, HdiffSmaller, *(beta, betamin, betamax))
    return beta, betamin, betamax, Hdiff

def binarySearch(res, el, Di, logU):
    print('Entered binary search function')
    Hdiff, thisP, beta, betamin, betamax = res
    beta, betamin, betamax, Hdiff = cond(np.abs(Hdiff) < 1e-5, lambda a, b, c, d: (a, b, c, d), HdiffGreaterTolerance, *(beta, betamin, betamax, Hdiff))
    (H, thisP) = Hbeta(Di, beta)
    Hdiff = H - logU
    return (Hdiff, thisP, beta, betamin, betamax), el

def x2beta_inner(Di: np.ndarray, iterator, beta, betamin, betamax, perplexity=30, tol=1e-5):
    """
    binary search for precision for Pi such that it matches the perplexity defined by the user
    :param Di: vector of length d-1, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: final probabilites p j|i
    """
    # Compute the Gaussian kernel and entropy for the current precision
    logU = np.log(perplexity)
    H, thisP = Hbeta(Di, beta)
    Hdiff = H - logU

    print('Starting binary search')
    binarySearch_func = partial(binarySearch, Di=Di, logU=logU)

    # Note: the following binary Search for suitable precisions (betas) will be repeated 50 times and does not include the threshold value
    (Hdiff, thisP, beta, betamin, betamax), el = scan(binarySearch_func, init=(Hdiff, thisP, beta, betamin, betamax), xs=None, length=1000)    # Set the final row of P
    #thisP = np.insert(thisP, iterator, 0)
    return beta

def x2beta(D: np.ndarray, tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values (high-dim space) in such a way that each
        conditional Gaussian has the same perplexity.
    """
    # Initialize some variables
    n = D.shape[0]
    beta = np.ones(n)      # precisions (1/sigma**2)
    betamin = np.full(n, -np.inf)
    betamax = np.full(n, np.inf)
    betas_final = vmap(partial(x2beta_inner, perplexity=perplexity, tol=tol))(D, np.arange(n), beta=beta, betamin=betamin, betamax=betamax)
    return betas_final

def x2distance(X):
    print("Computing pairwise distances...")
    sum_X = np.sum(np.square(X), 1)
    (n, d) = X.shape
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    D = np.reshape(np.delete(D, np.array([i for i in range(0, D.shape[0]**2, (D.shape[0]+1))])), (n , n - 1 ))
    return D

def distance2p(D, betas):
    P_final = vmap(Hbeta_final, in_axes=0)(D, betas)
    print('P_final', P_final, P_final.shape)
    P_final = vmap(partial(np.insert, values=0))(P_final, np.arange(P_final.shape[0]))
    return P_final

def y2q(Y: np.ndarray):
    # Compute pairwise affinities
    sum_Y = np.sum(np.square(Y), 1)
    num = -2. * np.dot(Y, Y.T)  # numerator
    num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
    num = num.at[np.diag_indices_from(num)].set(0.)     # numerator
    Q = num / np.sum(num)
    Q = np.maximum(Q, 1e-12)
    return Q, num

def KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    D = x2distance(X)
    print('D', D.shape)
    # first compute betas without tracking the derivative
    betas = x2beta(jax.lax.stop_gradient(D), tol=1e-5, perplexity=perplexity)
    print('betas', betas.shape, betas)
    # use final betas to compute the probability matrix from the distances.
    # here D and therefor X is tracked for derivative computation
    P = distance2p(D, betas)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    print('P', P, P.shape)
    Q, _ = y2q(Y)
    #print('Q', Q)
    return np.sum(P * (np.log(P+1e-10) - np.log(Q+1e-10)))

In [7]:
KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener)

Computing pairwise distances...
D (75, 74)
Starting binary search
Entered binary search function
betas (75,) [0.0833168  0.04818487 0.08273983 0.07380772 0.05245066 0.08294678
 0.0609293  0.05273724 0.09796333 0.03626108 0.08285427 0.05056906
 0.07479286 0.06325865 0.06680584 0.04696321 0.0906086  0.06389904
 0.07180119 0.06346607 0.04932213 0.05522728 0.06407213 0.05851746
 0.07254124 0.07178688 0.06860161 0.07835436 0.06815624 0.08107281
 0.05218029 0.06993198 0.0700655  0.05881786 0.04333305 0.05926228
 0.06289864 0.08408833 0.06576729 0.07532501 0.05331182 0.07644653
 0.04982805 0.08203888 0.05822277 0.05159092 0.05098009 0.0612545
 0.07812023 0.11333656 0.03504992 0.07473469 0.09666634 0.09368134
 0.06025791 0.06366062 0.07831383 0.07310867 0.03565502 0.06173611
 0.04958391 0.0692234  0.03981996 0.09915447 0.04073715 0.06004953
 0.06520844 0.08164024 0.09013748 0.07089806 0.03989506 0.08517838
 0.08844471 0.0534153  0.08211517]
P_final [[0.04358517 0.00039209 0.00387652 ... 0.0249

Array(0.36283004, dtype=float32)

In [8]:
J_X = jacfwd(KL_divergence, argnums=0)(X_flat, Y_flat, X_unflattener, Y_unflattener)
print('J', J_X)

Computing pairwise distances...
D (75, 74)
Starting binary search
Entered binary search function
betas (75,) [0.0833168  0.04818487 0.08273983 0.07380772 0.05245066 0.08294678
 0.0609293  0.05273724 0.09796333 0.03626108 0.08285427 0.05056906
 0.07479286 0.06325865 0.06680584 0.04696321 0.0906086  0.06389904
 0.07180119 0.06346607 0.04932213 0.05522728 0.06407213 0.05851746
 0.07254124 0.07178688 0.06860161 0.07835436 0.06815624 0.08107281
 0.05218029 0.06993198 0.0700655  0.05881786 0.04333305 0.05926228
 0.06289864 0.08408833 0.06576729 0.07532501 0.05331182 0.07644653
 0.04982805 0.08203888 0.05822277 0.05159092 0.05098009 0.0612545
 0.07812023 0.11333656 0.03504992 0.07473469 0.09666634 0.09368134
 0.06025791 0.06366062 0.07831383 0.07310867 0.03565502 0.06173611
 0.04958391 0.0692234  0.03981996 0.09915447 0.04073715 0.06004953
 0.06520844 0.08164024 0.09013748 0.07089806 0.03989506 0.08517838
 0.08844471 0.0534153  0.08211517]
P_final Traced<ConcreteArray([[0.04358517 0.00039209 

In [8]:
J_X_Y = jacfwd(jacfwd(KL_divergence, argnums=1), argnums=0)(X_flat, Y_flat, X_unflattener, Y_unflattener)
print('J', J_X_Y)

Computing pairwise distances...
D (100, 99)
Starting binary search
Entered binary search function
betas (100,) [0.10493088 0.05305099 0.06157207 0.06390667 0.06154919 0.07680702
 0.0699935  0.05819607 0.08342648 0.09432507 0.0731988  0.10660362
 0.06660604 0.06692028 0.04802561 0.10120201 0.06537056 0.06107044
 0.09982681 0.10559177 0.05676317 0.07765913 0.09629631 0.08690166
 0.08326244 0.05785751 0.08759308 0.11454678 0.06787348 0.05521774
 0.06664562 0.07845306 0.07565975 0.11841583 0.08559132 0.07411289
 0.08760452 0.07557774 0.08249092 0.10140514 0.09905815 0.06422043
 0.0360105  0.07557869 0.06585169 0.13054466 0.08201885 0.06791782
 0.09290886 0.11986828 0.07983208 0.06013155 0.08343124 0.0562501
 0.06475401 0.06837273 0.07059526 0.05203009 0.11010838 0.11074638
 0.07498837 0.06998587 0.07353783 0.05822563 0.09373283 0.06792021
 0.0934782  0.08785439 0.07491493 0.08535194 0.06063366 0.08080769
 0.06080723 0.05537701 0.06449699 0.09566116 0.08714008 0.10160828
 0.075284   0.06622

In [8]:
from simplified_tsne_jax import * 
def KL_divergence_sim(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    P = x2p(X, tol=1e-5, perplexity=perplexity)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    print('P', P)
    Q, _ = y2q(Y)
    #print('Q', Q)
    return np.sum(P * (np.log(P+1e-10) - np.log(Q+1e-10)))

In [9]:
KL_divergence_sim(X_flat, Y_flat, X_unflattener, Y_unflattener)

Computing pairwise distances...
Starting binary search
Entered binary search function
beta Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([0.03338718, 0.04345798, 0.04374123, 0.05164909, 0.03682137,
       0.04464912, 0.06151485, 0.07752228, 0.06982803, 0.03938293,
       0.04574871, 0.03265285, 0.04794598, 0.03188419, 0.04337311,
       0.03170156, 0.04881954, 0.05334282, 0.05403519, 0.03245258,
       0.04097939, 0.03977203, 0.03492451, 0.03466892, 0.0863533 ,
       0.07761765, 0.03615904, 0.06347084, 0.03732204, 0.04832458,
       0.03700161, 0.03839779, 0.05221367, 0.04576206, 0.04596138,
       0.07259178, 0.04757977, 0.0554266 , 0.03518295, 0.04201889,
       0.03783751, 0.05830765, 0.0630455 , 0.0828495 , 0.05039978,
       0.0639801 , 0.07695389, 0.04185009, 0.0306778 , 0.06483078],      dtype=float32)
  batch_dim = 0
P [[9.99999996e-13 2.90518976e-04 1.12769492e-04 ... 1.10822548e-04
  1.79829751e-03 3.63988620e-05]
 [2.90518976e-04 9.99999996e-1

Array(0.16367947, dtype=float32)

In [10]:
J_X = jacfwd(KL_divergence_sim, argnums=0)(X_flat, Y_flat, X_unflattener, Y_unflattener)
print('J', J_X)

Computing pairwise distances...
Starting binary search
Entered binary search function
beta Traced<ShapedArray(float32[])>with<BatchTrace(level=3/0)> with
  val = Array([0.03338718, 0.04345798, 0.04374123, 0.05164909, 0.03682137,
       0.04464912, 0.06151485, 0.07752228, 0.06982803, 0.03938293,
       0.04574871, 0.03265285, 0.04794598, 0.03188419, 0.04337311,
       0.03170156, 0.04881954, 0.05334282, 0.05403519, 0.03245258,
       0.04097939, 0.03977203, 0.03492451, 0.03466892, 0.0863533 ,
       0.07761765, 0.03615904, 0.06347084, 0.03732204, 0.04832458,
       0.03700161, 0.03839779, 0.05221367, 0.04576206, 0.04596138,
       0.07259178, 0.04757977, 0.0554266 , 0.03518295, 0.04201889,
       0.03783751, 0.05830765, 0.0630455 , 0.0828495 , 0.05039978,
       0.0639801 , 0.07695389, 0.04185009, 0.0306778 , 0.06483078],      dtype=float32)
  batch_dim = 0
P Traced<ConcreteArray([[9.99999996e-13 2.90518976e-04 1.12769492e-04 ... 1.10822548e-04
  1.79829751e-03 3.63988620e-05]
 [2.90518